In [1]:
import pykoi

/opt/conda/envs/pykoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define my DB

In [2]:
qa_database = pykoi.QuestionAnswerDatabase()

In [5]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [3]:
my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

,ID,Question,Answer,Vote Status,Timestamp
0,1,who founded YC and what does YC do,Y Combinator is a startup accelerator that pro...,down,2023-07-24 08:28:29.461045
1,2,what are the top YC companies,I am looking for a list of the top YC companie...,down,2023-07-24 08:29:17.521467
2,3,How much does top 10 YC companies worth,"According to the latest Crunchbase data, the 1...",down,2023-07-24 08:29:58.248998
3,4,What's the meaning of life,"The meaning of life is to be happy, to be usef...",down,2023-07-24 19:26:04.713297
4,5,What is the meaning of life,The meaning of life is to find your gift. The ...,up,2023-07-24 19:26:28.676427


In [6]:
my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
my_data_pd.shape

(118, 5)

### Load from HF

In [7]:
!wget https://huggingface.co/datasets/lvwerra/stack-exchange-paired/resolve/main/data/reward/train-00000-of-00020.parquet

--2023-07-26 22:30:10--  https://huggingface.co/datasets/lvwerra/stack-exchange-paired/resolve/main/data/reward/train-00000-of-00020.parquet
Resolving huggingface.co (huggingface.co)... 99.84.66.70, 99.84.66.72, 99.84.66.112, ...
Connecting to huggingface.co (huggingface.co)|99.84.66.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/da/9a/da9a97da0630d3286f74d6b44d86813e7485d35f9dc4dcf915108587a6c56338/1634d85b864804e51e92f1b9820eca64fbd34d5972d3eaed74a5dbef424f4494?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27train-00000-of-00020.parquet%3B+filename%3D%22train-00000-of-00020.parquet%22%3B&Expires=1690669810&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MDY2OTgxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9kYS85YS9kYTlhOTdkYTA2MzBkMzI4NmY3NGQ2YjQ0ZDg2ODEzZTc0ODVkMzVmOWRjNGRjZjkxNTEwODU4N2E2YzU2MzM4LzE2MzRkODViODY0ODA0ZTUxZTkyZjF

In [12]:
import pandas as pd

stack_exchange_rank_df = pd.read_parquet('train-00000-of-00020.parquet')
stack_exchange_rank_df

,qid,question,date,metadata,response_j,response_k
0,538868,I have a question to those who are really prof...,2020/03/27,[https://physics.stackexchange.com/questions/5...,In the absence of a magnetic field the velocit...,"Can MHD work on ""cold plasma""? Is there any ef..."
1,3665283,"When looking at dI/dt we have a term gamma\*I,...",2020/05/08,[https://math.stackexchange.com/questions/3665...,If we have $I$ infected people and the recover...,Imagine a situation where each infected indivi...
2,3665283,"When looking at dI/dt we have a term gamma\*I,...",2020/05/08,[https://math.stackexchange.com/questions/3665...,If we have $I$ infected people and the recover...,"The others have addressed the question, I just..."
3,3665283,"When looking at dI/dt we have a term gamma\*I,...",2020/05/08,[https://math.stackexchange.com/questions/3665...,Imagine a situation where each infected indivi...,"The others have addressed the question, I just..."
4,32965609,I have a setup where I use a service to log a ...,2015/10/06,"[https://Stackoverflow.com/questions/32965609,...","In your code, if there is no user logged in (s...",My current work-around is creating an other Sa...
...,...,...,...,...,...,...
372095,55926830,I'm filtering some data where say table.x has ...,2019/04/30,"[https://Stackoverflow.com/questions/55926830,...",What about using `DISTINCT` with one column li...,Try to use window function\n\n```\nselect \n ...
372096,55926830,I'm filtering some data where say table.x has ...,2019/04/30,"[https://Stackoverflow.com/questions/55926830,...",You could usea inner join on max col3 group by...,Try to use window function\n\n```\nselect \n ...
372097,55926830,I'm filtering some data where say table.x has ...,2019/04/30,"[https://Stackoverflow.com/questions/55926830,...",Try the following SQL code:\n\n```\nSELECT max...,Try to use window function\n\n```\nselect \n ...
372098,282381,I want to return the previous row based on the...,2020/12/31,[https://dba.stackexchange.com/questions/28238...,Window functions work on the result after the ...,You just need to add an `OVER` and `ORDER BY` ...


In [13]:
stack_exchange_rank_df = stack_exchange_rank_df[['qid','question', 'response_j', 'response_k']]
stack_exchange_rank_df.head()

,qid,question,response_j,response_k
0,538868,I have a question to those who are really prof...,In the absence of a magnetic field the velocit...,"Can MHD work on ""cold plasma""? Is there any ef..."
1,3665283,"When looking at dI/dt we have a term gamma\*I,...",If we have $I$ infected people and the recover...,Imagine a situation where each infected indivi...
2,3665283,"When looking at dI/dt we have a term gamma\*I,...",If we have $I$ infected people and the recover...,"The others have addressed the question, I just..."
3,3665283,"When looking at dI/dt we have a term gamma\*I,...",Imagine a situation where each infected indivi...,"The others have addressed the question, I just..."
4,32965609,I have a setup where I use a service to log a ...,"In your code, if there is no user logged in (s...",My current work-around is creating an other Sa...


In [14]:
stack_exchange_rank_df.columns = RANKING_CSV_HEADER
stack_exchange_rank_df.head()

,ID,Question,Up Ranking Answer,Low Ranking Answer
0,538868,I have a question to those who are really prof...,In the absence of a magnetic field the velocit...,"Can MHD work on ""cold plasma""? Is there any ef..."
1,3665283,"When looking at dI/dt we have a term gamma\*I,...",If we have $I$ infected people and the recover...,Imagine a situation where each infected indivi...
2,3665283,"When looking at dI/dt we have a term gamma\*I,...",If we have $I$ infected people and the recover...,"The others have addressed the question, I just..."
3,3665283,"When looking at dI/dt we have a term gamma\*I,...",Imagine a situation where each infected indivi...,"The others have addressed the question, I just..."
4,32965609,I have a setup where I use a service to log a ...,"In your code, if there is no user logged in (s...",My current work-around is creating an other Sa...


In [15]:
stack_exchange_rank_df.head(10000).to_csv("stack_exchange_rank_10k_dataset.csv")

In [17]:
from datasets import Dataset, load_dataset
dataset = Dataset.from_pandas(stack_exchange_rank_df.head(10000))
dataset

Dataset({
    features: ['ID', 'Question', 'Up Ranking Answer', 'Low Ranking Answer'],
    num_rows: 10000
})

In [21]:
## On HF Hub UI, create a new Dataset (source: https://huggingface.co/docs/datasets/upload_dataset#create-a-repository)
dataset.push_to_hub("goldmermaid/stack_exchange_rank_10k_dataset", private=True)


Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.87s/it]
